### Web Scrapping


* direccion de hotel
* atributos
* descripcion
* reviews
* latitud y longitud del mapa

In [19]:
import time
import pandas as pd
import geopandas as gpd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

el scrap de atributos ya modificado y optimizado

In [8]:
def scrap_hotels_attributes(urls):

    hotels_attributes = pd.DataFrame()

    # Instanciate and configurate driver
    chrome_options = Options()
    chrome_options.add_argument('--headless') # unables GUI
    chrome_options.add_argument('--disable-infobars') # unables images loading
    chrome_options.add_argument('--blink-settings=imagesEnabled=false')  # deshabilita carga de imágenes
    driver = webdriver.Chrome(options=chrome_options)
    
    for i, url in enumerate(urls):

        driver.get(url)
        print(f"{i+1}/{len(urls)}")

        # Obtener dirección
        try:
            direccion_element = driver.find_element(By.CLASS_NAME, "hp_address_subtitle")
            direccion_texto = direccion_element.text
        except:
            direccion_texto = None

        # Obtener descripción
        try:
            descripcion_element = driver.find_element(By.CLASS_NAME, "b3efd73f69")
            descripcion_texto = descripcion_element.text
        except:
            descripcion_texto = None

        # Obtener atributos
        try:
            list_items = driver.find_elements(By.XPATH, "/html/body/div[3]/div/div[5]/div[1]/div[1]/div[3]/div/div/div[1]/div[2]/div[2]/div/div/ul/li")
            attributes_list = [li.text for li in list_items]
        except:
            attributes_list = None

        # Obtener latitud y longitud
        try:
            map_element = driver.find_element(By.ID, 'hotel_sidebar_static_map')
            lat_lng_attribute = map_element.get_attribute('data-atlas-latlng')
            lat, lng = map(float, lat_lng_attribute.split(','))
        except:
            lat, lng = None, None

        # Scores
        score_list = []
        try:
            scores = driver.find_elements(By.CLASS_NAME, 'b817090550')
            for i, score in enumerate(scores):
                score = score.text
                score = score.replace(",", ".")
                parts = score.split("\n")
                if len(parts) == 2:
                    score_list.append({f"{parts[0]}": parts[1]})
        except:
            pass

        # Crear el DataFrame
        attributes = pd.DataFrame({
            "Dirección": [direccion_texto],
            "Descripción": [descripcion_texto],
            "Atributos": [attributes_list],
            "Latitud": [lat],
            "Longitud": [lng],
            "Scores": [score_list]
        })

        hotels_attributes = pd.concat([hotels_attributes, attributes], ignore_index=True)

    driver.quit()
    return hotels_attributes

# Example
utah_hotels = pd.read_csv(r"data\utah_hotels.csv")
utah_hotels_attributes = scrap_hotels_attributes(utah_hotels["reviews_url"][700:].tolist())
utah_hotels_attributes.to_csv('utah_hotels_attributes_parte02.csv', index=False)

1/593
2/593
3/593
4/593
5/593
6/593
7/593
8/593
9/593
10/593
11/593
12/593
13/593
14/593
15/593
16/593
17/593
18/593
19/593
20/593
21/593
22/593
23/593
24/593
25/593
26/593
27/593
28/593
29/593
30/593
31/593
32/593
33/593
34/593
35/593
36/593
37/593
38/593
39/593
40/593
41/593
42/593
43/593
44/593
45/593
46/593
47/593
48/593
49/593
50/593
51/593
52/593
53/593
54/593
55/593
56/593
57/593
58/593
59/593
60/593
61/593
62/593
63/593
64/593
65/593
66/593
67/593
68/593
69/593
70/593
71/593
72/593
73/593
74/593
75/593
76/593
77/593
78/593
79/593
80/593
81/593
82/593
83/593
84/593
85/593
86/593
87/593
88/593
89/593
90/593
91/593
92/593
93/593
94/593
95/593
96/593
97/593
98/593
99/593
100/593
101/593
102/593
103/593
104/593
105/593
106/593
107/593
108/593
109/593
110/593
111/593
112/593
113/593
114/593
115/593
116/593
117/593
118/593
119/593
120/593
121/593
122/593
123/593
124/593
125/593
126/593
127/593
128/593
129/593
130/593
131/593
132/593
133/593
134/593
135/593
136/593
137/593
138/593
139/

In [62]:
def re_scrape_attributes(urls):
    # Instanciate and configurate driver
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # unables GUI
    chrome_options.add_argument('--disable-infobars')  # unables images loading
    chrome_options.add_argument('--blink-settings=imagesEnabled=false')  # deshabilita carga de imágenes
    driver = webdriver.Chrome(options=chrome_options)

    # Nueva columna de atributos
    new_attributes_list = []

    for i, url in enumerate(urls):
        driver.get(url)
        print(f"{i + 1}/{len(urls)}")

        # Obtener atributos
        try:
            list_items = driver.find_elements(By.XPATH,
                                              "/html/body/div[3]/div/div[6]/div[1]/div[1]/div[3]/div/div/div[1]/div[2]/div[2]/div/div/ul/li")
            attributes_list = [li.text for li in list_items]
            new_attributes_list.append(attributes_list)
        except:
            new_attributes_list.append(None)

    driver.quit()

    # Crear un nuevo DataFrame solo con la columna de atributos
    new_attributes_df = pd.DataFrame({"Atributos": new_attributes_list})

    return new_attributes_df

##### Florida

In [22]:
florida_hotels = pd.read_csv(r"data\raw_Florida_hotels.csv")

In [61]:
# Instanciate and configurate driver
chrome_options = Options()
#chrome_options.add_argument('--headless') # unables GUI
chrome_options.add_argument('--disable-infobars') # unables images loading
driver = webdriver.Chrome(options=chrome_options)

url = florida_hotels["url"][1]
driver.get(url)
time.sleep(5)
#              /html/body/div[3]/div/div[5]/div[1]/div[1]/div[4]/div/div/div[1]/div[2]/div[2]/div/div/ul
#xpath florida /html/body/div[3]/div/div[6]/div[1]/div[1]/div[3]/div/div/div[1]/div[2]/div[2]/div/div/ul/li
#xpath utah    /html/body/div[3]/div/div[5]/div[1]/div[1]/div[3]/div/div/div[1]/div[2]/div[2]/div/div/ul/li
# Encontrar los elementos li directamente
list_items = driver.find_elements(By.XPATH, "/html/body/div[3]/div/div[6]/div[1]/div[1]/div[3]/div/div/div[1]/div[2]/div[2]/div/div/ul/li")

# Crear una lista para guardar los atributos
attributes_list = []

for li in list_items:
    # Extraer el contenido de los elementos li
    attributes_list.append(li.text)

print(attributes_list)

# Cerrar el navegador
driver.quit()

['2 piscinas', 'Habitaciones sin humo', 'Gimnasio', '2 restaurantes', 'Adaptado personas de movilidad reducida', 'Habitaciones familiares', 'Situado frente a la playa', 'Spa y centro de bienestar', 'Servicio de habitaciones', 'Bar']


In [52]:
# cargo el dataset en un df
florida_hotels = pd.read_csv(r"data\raw_Florida_hotels.csv")

In [69]:
florida_hotels['url'][3000]

'https://www.booking.com/hotel/us/tropical-fountains-condo.es.html?label=bin859jc-1DCAMo7AE4mgNIClgDaAyIAQGYAQq4ARfIAQzYAQPoAQH4AQKIAgGoAgO4AtamjqsGwAIB0gIkNjY3NWU0MzUtZDQ2Yy00MGI3LWE3M2ItYzQ5YTQ3YmJhY2M52AIE4AIB&sid=867675c0a6ac4ae450c754b639211fc5&aid=357028&ucfs=1&arphpl=1&checkin=2024-05-12&checkout=2024-05-18&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=18&hapos=318&sr_order=popularity&srpvid=8fa566ccac1d0073&srepoch=1701095869&all_sr_blocks=951773902_367772170_6_0_0&highlighted_blocks=951773902_367772170_6_0_0&matching_block_id=951773902_367772170_6_0_0&sr_pri_blocks=951773902_367772170_6_0_0__87363&from=searchresults#hotelTmpl'

In [63]:
# Example Primera parte
florida_hotels_attributes = re_scrape_attributes(florida_hotels["url"][:1000].tolist())
florida_hotels_attributes.to_csv('florida_hotels_attributes_parte01.csv', index=False)

1/1000
2/1000
3/1000
4/1000
5/1000
6/1000
7/1000
8/1000
9/1000
10/1000
11/1000
12/1000
13/1000
14/1000
15/1000
16/1000
17/1000
18/1000
19/1000
20/1000
21/1000
22/1000
23/1000
24/1000
25/1000
26/1000
27/1000
28/1000
29/1000
30/1000
31/1000
32/1000
33/1000
34/1000
35/1000
36/1000
37/1000
38/1000
39/1000
40/1000
41/1000
42/1000
43/1000
44/1000
45/1000
46/1000
47/1000
48/1000
49/1000
50/1000
51/1000
52/1000
53/1000
54/1000
55/1000
56/1000
57/1000
58/1000
59/1000
60/1000
61/1000
62/1000
63/1000
64/1000
65/1000
66/1000
67/1000
68/1000
69/1000
70/1000
71/1000
72/1000
73/1000
74/1000
75/1000
76/1000
77/1000
78/1000
79/1000
80/1000
81/1000
82/1000
83/1000
84/1000
85/1000
86/1000
87/1000
88/1000
89/1000
90/1000
91/1000
92/1000
93/1000
94/1000
95/1000
96/1000
97/1000
98/1000
99/1000
100/1000
101/1000
102/1000
103/1000
104/1000
105/1000
106/1000
107/1000
108/1000
109/1000
110/1000
111/1000
112/1000
113/1000
114/1000
115/1000
116/1000
117/1000
118/1000
119/1000
120/1000
121/1000
122/1000
123/1000
1

In [64]:
# Segunda parte
florida_hotels_attributes_02 = re_scrape_attributes(florida_hotels["url"][1000:2000].tolist())
florida_hotels_attributes_02.to_csv('florida_hotels_attributes_parte02.csv', index=False)

1/1000
2/1000
3/1000
4/1000
5/1000
6/1000
7/1000
8/1000
9/1000
10/1000
11/1000
12/1000
13/1000
14/1000
15/1000
16/1000
17/1000
18/1000
19/1000
20/1000
21/1000
22/1000
23/1000
24/1000
25/1000
26/1000
27/1000
28/1000
29/1000
30/1000
31/1000
32/1000
33/1000
34/1000
35/1000
36/1000
37/1000
38/1000
39/1000
40/1000
41/1000
42/1000
43/1000
44/1000
45/1000
46/1000
47/1000
48/1000
49/1000
50/1000
51/1000
52/1000
53/1000
54/1000
55/1000
56/1000
57/1000
58/1000
59/1000
60/1000
61/1000
62/1000
63/1000
64/1000
65/1000
66/1000
67/1000
68/1000
69/1000
70/1000
71/1000
72/1000
73/1000
74/1000
75/1000
76/1000
77/1000
78/1000
79/1000
80/1000
81/1000
82/1000
83/1000
84/1000
85/1000
86/1000
87/1000
88/1000
89/1000
90/1000
91/1000
92/1000
93/1000
94/1000
95/1000
96/1000
97/1000
98/1000
99/1000
100/1000
101/1000
102/1000
103/1000
104/1000
105/1000
106/1000
107/1000
108/1000
109/1000
110/1000
111/1000
112/1000
113/1000
114/1000
115/1000
116/1000
117/1000
118/1000
119/1000
120/1000
121/1000
122/1000
123/1000
1

In [65]:
# Tercera parte
florida_hotels_attributes_03 = re_scrape_attributes(florida_hotels["url"][2000:3000].tolist())
florida_hotels_attributes_03.to_csv('florida_hotels_attributes_parte03.csv', index=False)

1/1000
2/1000
3/1000
4/1000
5/1000
6/1000
7/1000
8/1000
9/1000
10/1000
11/1000
12/1000
13/1000
14/1000
15/1000
16/1000
17/1000
18/1000
19/1000
20/1000
21/1000
22/1000
23/1000
24/1000
25/1000
26/1000
27/1000
28/1000
29/1000
30/1000
31/1000
32/1000
33/1000
34/1000
35/1000
36/1000
37/1000
38/1000
39/1000
40/1000
41/1000
42/1000
43/1000
44/1000
45/1000
46/1000
47/1000
48/1000
49/1000
50/1000
51/1000
52/1000
53/1000
54/1000
55/1000
56/1000
57/1000
58/1000
59/1000
60/1000
61/1000
62/1000
63/1000
64/1000
65/1000
66/1000
67/1000
68/1000
69/1000
70/1000
71/1000
72/1000
73/1000
74/1000
75/1000
76/1000
77/1000
78/1000
79/1000
80/1000
81/1000
82/1000
83/1000
84/1000
85/1000
86/1000
87/1000
88/1000
89/1000
90/1000
91/1000
92/1000
93/1000
94/1000
95/1000
96/1000
97/1000
98/1000
99/1000
100/1000
101/1000
102/1000
103/1000
104/1000
105/1000
106/1000
107/1000
108/1000
109/1000
110/1000
111/1000
112/1000
113/1000
114/1000
115/1000
116/1000
117/1000
118/1000
119/1000
120/1000
121/1000
122/1000
123/1000
1

In [66]:
# Cuarta parte
florida_hotels_attributes_04 = re_scrape_attributes(florida_hotels["url"][3000:4000].tolist())
florida_hotels_attributes_04.to_csv('florida_hotels_attributes_parte04.csv', index=False)

1/1000
2/1000
3/1000
4/1000
5/1000
6/1000
7/1000
8/1000
9/1000
10/1000
11/1000
12/1000
13/1000
14/1000
15/1000
16/1000
17/1000
18/1000
19/1000
20/1000
21/1000
22/1000
23/1000
24/1000
25/1000
26/1000
27/1000
28/1000
29/1000
30/1000
31/1000
32/1000
33/1000
34/1000
35/1000
36/1000
37/1000
38/1000
39/1000
40/1000
41/1000
42/1000
43/1000
44/1000
45/1000
46/1000
47/1000
48/1000
49/1000
50/1000
51/1000
52/1000
53/1000
54/1000
55/1000
56/1000
57/1000
58/1000
59/1000
60/1000
61/1000
62/1000
63/1000
64/1000
65/1000
66/1000
67/1000
68/1000
69/1000
70/1000
71/1000
72/1000
73/1000
74/1000
75/1000
76/1000
77/1000
78/1000
79/1000
80/1000
81/1000
82/1000
83/1000
84/1000
85/1000
86/1000
87/1000
88/1000
89/1000
90/1000
91/1000
92/1000
93/1000
94/1000
95/1000
96/1000
97/1000
98/1000
99/1000
100/1000
101/1000
102/1000
103/1000
104/1000
105/1000
106/1000
107/1000
108/1000
109/1000
110/1000
111/1000
112/1000
113/1000
114/1000
115/1000
116/1000
117/1000
118/1000
119/1000
120/1000
121/1000
122/1000
123/1000
1

In [67]:
# Quinta parte
florida_hotels_attributes_05 = re_scrape_attributes(florida_hotels["url"][4000:5000].tolist())
florida_hotels_attributes_05.to_csv('florida_hotels_attributes_parte05.csv', index=False)

1/1000
2/1000
3/1000
4/1000
5/1000
6/1000
7/1000
8/1000
9/1000
10/1000
11/1000
12/1000
13/1000
14/1000
15/1000
16/1000
17/1000
18/1000
19/1000
20/1000
21/1000
22/1000
23/1000
24/1000
25/1000
26/1000
27/1000
28/1000
29/1000
30/1000
31/1000
32/1000
33/1000
34/1000
35/1000
36/1000
37/1000
38/1000
39/1000
40/1000
41/1000
42/1000
43/1000
44/1000
45/1000
46/1000
47/1000
48/1000
49/1000
50/1000
51/1000
52/1000
53/1000
54/1000
55/1000
56/1000
57/1000
58/1000
59/1000
60/1000
61/1000
62/1000
63/1000
64/1000
65/1000
66/1000
67/1000
68/1000
69/1000
70/1000
71/1000
72/1000
73/1000
74/1000
75/1000
76/1000
77/1000
78/1000
79/1000
80/1000
81/1000
82/1000
83/1000
84/1000
85/1000
86/1000
87/1000
88/1000
89/1000
90/1000
91/1000
92/1000
93/1000
94/1000
95/1000
96/1000
97/1000
98/1000
99/1000
100/1000
101/1000
102/1000
103/1000
104/1000
105/1000
106/1000
107/1000
108/1000
109/1000
110/1000
111/1000
112/1000
113/1000
114/1000
115/1000
116/1000
117/1000
118/1000
119/1000
120/1000
121/1000
122/1000
123/1000
1

In [70]:
# Sexta parte
florida_hotels_attributes_06 = re_scrape_attributes(florida_hotels["url"][5000:6000].tolist())
florida_hotels_attributes_06.to_csv('florida_hotels_attributes_parte06.csv', index=False)

1/1000
2/1000
3/1000
4/1000
5/1000
6/1000
7/1000
8/1000
9/1000
10/1000
11/1000
12/1000
13/1000
14/1000
15/1000
16/1000
17/1000
18/1000
19/1000
20/1000
21/1000
22/1000
23/1000
24/1000
25/1000
26/1000
27/1000
28/1000
29/1000
30/1000
31/1000
32/1000
33/1000
34/1000
35/1000
36/1000
37/1000
38/1000
39/1000
40/1000
41/1000
42/1000
43/1000
44/1000
45/1000
46/1000
47/1000
48/1000
49/1000
50/1000
51/1000
52/1000
53/1000
54/1000
55/1000
56/1000
57/1000
58/1000
59/1000
60/1000
61/1000
62/1000
63/1000
64/1000
65/1000
66/1000
67/1000
68/1000
69/1000
70/1000
71/1000
72/1000
73/1000
74/1000
75/1000
76/1000
77/1000
78/1000
79/1000
80/1000
81/1000
82/1000
83/1000
84/1000
85/1000
86/1000
87/1000
88/1000
89/1000
90/1000
91/1000
92/1000
93/1000
94/1000
95/1000
96/1000
97/1000
98/1000
99/1000
100/1000
101/1000
102/1000
103/1000
104/1000
105/1000
106/1000
107/1000
108/1000
109/1000
110/1000
111/1000
112/1000
113/1000
114/1000
115/1000
116/1000
117/1000
118/1000
119/1000
120/1000
121/1000
122/1000
123/1000
1

In [ ]:
# Séptima parte
florida_hotels_attributes_07 = re_scrape_attributes(florida_hotels["url"][6000:7000].tolist())
florida_hotels_attributes_07.to_csv('florida_hotels_attributes_parte07.csv', index=False)

1/1000
2/1000
3/1000
4/1000
5/1000
6/1000
7/1000
8/1000
9/1000
10/1000
11/1000
12/1000
13/1000
14/1000
15/1000
16/1000
17/1000
18/1000
19/1000
20/1000
21/1000
22/1000
23/1000
24/1000
25/1000
26/1000
27/1000
28/1000
29/1000
30/1000
31/1000
32/1000
33/1000
34/1000
35/1000
36/1000
37/1000
38/1000
39/1000
40/1000
41/1000
42/1000
43/1000
44/1000
45/1000
46/1000
47/1000
48/1000
49/1000
50/1000
51/1000
52/1000
53/1000
54/1000
55/1000
56/1000
57/1000
58/1000
59/1000
60/1000
61/1000
62/1000
63/1000
64/1000
65/1000
66/1000
67/1000
68/1000
69/1000
70/1000
71/1000
72/1000
73/1000
74/1000
75/1000
76/1000
77/1000
78/1000
79/1000
80/1000
81/1000
82/1000
83/1000
84/1000
85/1000
86/1000
87/1000
88/1000
89/1000
90/1000
91/1000
92/1000
93/1000
94/1000
95/1000
96/1000
97/1000
98/1000
99/1000
100/1000
101/1000
102/1000
103/1000
104/1000
105/1000
106/1000
107/1000
108/1000
109/1000
110/1000
111/1000
112/1000
113/1000
114/1000
115/1000
116/1000
117/1000
118/1000
119/1000
120/1000
121/1000
122/1000
123/1000
1

In [ ]:
# Octava parte
florida_hotels_attributes_08 = re_scrape_attributes(florida_hotels["url"][7000:8000].tolist())
florida_hotels_attributes_08.to_csv('florida_hotels_attributes_parte08.csv', index=False)

1/1000
2/1000
3/1000
4/1000
5/1000
6/1000
7/1000
8/1000
9/1000
10/1000
11/1000
12/1000
13/1000
14/1000
15/1000
16/1000
17/1000
18/1000
19/1000
20/1000
21/1000
22/1000
23/1000
24/1000
25/1000
26/1000
27/1000
28/1000
29/1000
30/1000
31/1000
32/1000
33/1000
34/1000
35/1000
36/1000
37/1000
38/1000
39/1000
40/1000
41/1000
42/1000
43/1000
44/1000
45/1000
46/1000
47/1000
48/1000
49/1000
50/1000
51/1000
52/1000
53/1000
54/1000
55/1000
56/1000
57/1000
58/1000
59/1000
60/1000
61/1000
62/1000
63/1000
64/1000
65/1000
66/1000
67/1000
68/1000
69/1000
70/1000
71/1000
72/1000
73/1000
74/1000
75/1000
76/1000
77/1000
78/1000
79/1000
80/1000
81/1000
82/1000
83/1000
84/1000
85/1000
86/1000
87/1000
88/1000
89/1000
90/1000
91/1000
92/1000
93/1000
94/1000
95/1000
96/1000
97/1000
98/1000
99/1000
100/1000
101/1000
102/1000
103/1000
104/1000
105/1000
106/1000
107/1000
108/1000
109/1000
110/1000
111/1000
112/1000
113/1000
114/1000
115/1000
116/1000
117/1000
118/1000
119/1000
120/1000
121/1000
122/1000
123/1000
1

In [ ]:
# Novena parte
florida_hotels_attributes_09 = re_scrape_attributes(florida_hotels["url"][8000:9000].tolist())
florida_hotels_attributes_09.to_csv('florida_hotels_attributes_parte09.csv', index=False)

1/1000
2/1000
3/1000
4/1000
5/1000
6/1000
7/1000
8/1000
9/1000
10/1000
11/1000
12/1000
13/1000
14/1000
15/1000
16/1000
17/1000
18/1000
19/1000
20/1000
21/1000
22/1000
23/1000
24/1000
25/1000
26/1000
27/1000
28/1000
29/1000
30/1000
31/1000
32/1000
33/1000
34/1000
35/1000
36/1000
37/1000
38/1000
39/1000
40/1000
41/1000
42/1000
43/1000
44/1000
45/1000
46/1000
47/1000
48/1000
49/1000
50/1000
51/1000
52/1000
53/1000
54/1000
55/1000
56/1000
57/1000
58/1000
59/1000
60/1000
61/1000
62/1000
63/1000
64/1000
65/1000
66/1000
67/1000
68/1000
69/1000
70/1000
71/1000
72/1000
73/1000
74/1000
75/1000
76/1000
77/1000
78/1000
79/1000
80/1000
81/1000
82/1000
83/1000
84/1000
85/1000
86/1000
87/1000
88/1000
89/1000
90/1000
91/1000
92/1000
93/1000
94/1000
95/1000
96/1000
97/1000
98/1000
99/1000
100/1000
101/1000
102/1000
103/1000
104/1000
105/1000
106/1000
107/1000
108/1000
109/1000
110/1000
111/1000
112/1000
113/1000
114/1000
115/1000
116/1000
117/1000
118/1000
119/1000
120/1000
121/1000
122/1000
123/1000
1

In [ ]:
# Décima parte
florida_hotels_attributes_10 = re_scrape_attributes(florida_hotels["url"][9000:].tolist())
florida_hotels_attributes_10.to_csv('florida_hotels_attributes_parte10.csv', index=False)

1/1356
2/1356
3/1356
4/1356
5/1356
6/1356
7/1356
8/1356
9/1356
10/1356
11/1356
12/1356
13/1356
14/1356
15/1356
16/1356
17/1356
18/1356
19/1356
20/1356
21/1356
22/1356
23/1356
24/1356
25/1356
26/1356
27/1356
28/1356
29/1356
30/1356
31/1356
32/1356
33/1356
34/1356
35/1356
36/1356
37/1356
38/1356
39/1356
40/1356
41/1356
42/1356
43/1356
44/1356
45/1356
46/1356
47/1356
48/1356
49/1356
50/1356
51/1356
52/1356
53/1356
54/1356
55/1356
56/1356
57/1356
58/1356
59/1356
60/1356
61/1356
62/1356
63/1356
64/1356
65/1356
66/1356
67/1356
68/1356
69/1356
70/1356
71/1356
72/1356
73/1356
74/1356
75/1356
76/1356
77/1356
78/1356
79/1356
80/1356
81/1356
82/1356
83/1356
84/1356
85/1356
86/1356
87/1356
88/1356
89/1356
90/1356
91/1356
92/1356
93/1356
94/1356
95/1356
96/1356
97/1356
98/1356
99/1356
100/1356
101/1356
102/1356
103/1356
104/1356
105/1356
106/1356
107/1356
108/1356
109/1356
110/1356
111/1356
112/1356
113/1356
114/1356
115/1356
116/1356
117/1356
118/1356
119/1356
120/1356
121/1356
122/1356
123/1356
1

In [39]:
import glob

# Obtén la lista de archivos CSV en el directorio
archivos_csv = glob.glob("*.csv")

print(archivos_csv)

['florida_hotels_attributes_parte01.csv', 'florida_hotels_attributes_parte02.csv', 'florida_hotels_attributes_parte03.csv', 'florida_hotels_attributes_parte04.csv', 'florida_hotels_attributes_parte05.csv', 'florida_hotels_attributes_parte06.csv', 'florida_hotels_attributes_parte07.csv', 'florida_hotels_attributes_parte08.csv', 'florida_hotels_attributes_parte09.csv', 'florida_hotels_attributes_parte10.csv']


In [40]:
# Lee cada archivo CSV y concaténalos en un DataFrame
dataframes = [pd.read_csv(archivo) for archivo in archivos_csv]
df_final = pd.concat(dataframes, ignore_index=True)

# Guarda el DataFrame resultante en un nuevo archivo CSV
df_final.to_csv("archivo_final.csv", index=False)

In [48]:
df_final

,Dirección,Descripción,Atributos,Latitud,Longitud,Scores
0,"3101 Collins Avenue, Miami Beach, FL 33140, Es...","Este hotel se encuentra en Miami Beach, a 160 ...",[],25.806778,-80.124064,[{'Habitación Deluxe con cama extragrande y vi...
1,"1440 Ocean Drive, Miami Beach, FL 33139, Estad...","El The Betsy Hotel, South Beach está ubicado e...",[],25.786503,-80.129663,[{'Habitación con cama extragrande.': '1 cama ...
2,"1601 Collins Avenue, Miami Beach, FL 33139, Es...",Este complejo de Miami Beach está situado fren...,[],25.789342,-80.130044,"[{'Personal': '8.6'}, {'Instalaciones y servic..."
3,"1650 James Avenue, Miami Beach, FL 33139, Esta...",Este hotel de estilo Art Déco está a 4 minutos...,[],25.791038,-80.130882,"[{'Habitación Doble Petite': '1 cama doble'}, ..."
4,"18683 Collins Avenue, Miami Beach, FL 33160, E...",Este hotel costero de Sunny Isles Beach se enc...,[],25.949090,-80.120217,[{'Suite de 1 dormitorio': '1 cama doble extra...
...,...,...,...,...,...,...
10351,"4111 South Ocean Drive, Hollywood, 33019, Esta...",El Luxury Condo Hotel High Ceiling Ocean Front...,[],25.986807,-80.118652,[]
10352,"Hollywood, 33019, Estados Unidos",El Bella Vista Getaway se encuentra en Hollywo...,[],26.027911,-80.115269,"[{'Personal': '10'}, {'Instalaciones y servici..."
10353,"5731 Southwest 54th Terrace, Davie, 33314, Est...","El New Villa, Heated Pool, Near Hard Rock 4BR3...",['WiFi gratis'],26.049210,-80.219420,"[{'Personal': '10'}, {'Instalaciones y servici..."
10354,"1341 Northeast 15th Avenue, Fort Lauderdale, 3...",El Home at Poinsettia Heights es un establecim...,[],26.144096,-80.129143,"[{'Personal': '7.5'}, {'Instalaciones y servic..."


### Reviews

Este es el fallido 

In [66]:
# Instanciate and configurate driver
chrome_options = Options()
#chrome_options.add_argument('--headless') # unables GUI
chrome_options.add_argument('--disable-infobars') # unables images loading
driver = webdriver.Chrome(options=chrome_options)

url = data["url"][0]
driver.get(url)
time.sleep(5)

# Hacer clic en el botón "Leer todos los comentarios"
read_all_reviews_button = driver.find_element(By.XPATH, "//span[text()='Leer todos los comentarios']")
read_all_reviews_button.click()
time.sleep(5)  # Esperar a que se carguen todas las reseñas

# Lista para almacenar los datos
data = []


# Contador de clics en el botón de la página siguiente
click_count = 0

# Loop para navegar por las páginas de reseñas
while click_count < 3:
    # Encontrar los elementos que contienen las reseñas en la página actual
    review_items = driver.find_elements(By.CLASS_NAME, "bui-grid__column-9")

    # Procesar las reseñas de la página actual
    for review_item in review_items:
        try:
            # Extraer información específica de la reseña
            review_date = review_item.find_elements(By.CLASS_NAME, "c-review-block__date").text
            review_title = review_item.find_elements(By.CLASS_NAME, "c-review-block__title").text
            review_score = review_item.find_elements(By.CLASS_NAME, "bui-review-score__badge").text
            positive_comment = review_item.find_elements(By.CLASS_NAME, "c-review__prefix--color-green").find_element(By.CLASS_NAME, "c-review__body").text
            negative_comment = review_item.find_elements(By.CLASS_NAME, "c-review__prefix").find_element(By.CLASS_NAME, "c-review__body").text
            
            # Verificar si hay una respuesta del hotel
            response_element = review_item.find_element(By.CLASS_NAME, "c-review-block__response")
            if response_element:
                response_body = response_element.find_element(By.CLASS_NAME, "c-review-block__response__body").text
            else:
                response_body = None
            
            
            # Agregar la información a la lista
            data.append({
                "Fecha de la reseña": review_date,
                "Título de la reseña": review_title,
                "Puntuación de la reseña": review_score,
                "Comentario positivo": positive_comment,
                "Comentario negativo": negative_comment,
                "Respuesta del hotel (Cuerpo)": response_body
            })
        except NoSuchElementException as e:
            print(f"Error al extraer información de la reseña: {e}")
    # Incrementar el contador de clics
    click_count += 1

  # Intentar encontrar el botón de la página siguiente
    try:
        next_page_button = driver.find_element(By.XPATH, "//a[@class='pagenext']")
        
        # Hacer clic en el botón de la página siguiente
        next_page_button.click()
        time.sleep(5)  # Esperar a que la nueva página de reseñas se cargue completamente
    except NoSuchElementException:
        break  # Salir del bucle si no se encuentra el botón de página siguiente
    except StaleElementReferenceException:
        # Si se produce un StaleElementReferenceException, vuelve a buscar el botón
        next_page_button = driver.find_element(By.XPATH, "//a[@class='pagenext']")
        next_page_button.click()
        time.sleep(5)  # Esperar a que la nueva página de reseñas se cargue completamente


Error al extraer información de la reseña: Message: no such element: Unable to locate element: {"method":"css selector","selector":".c-review-block__response"}
  (Session info: chrome=119.0.6045.160); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6425E82B2+55298]
	(No symbol) [0x00007FF642555E02]
	(No symbol) [0x00007FF6424105AB]
	(No symbol) [0x00007FF64245175C]
	(No symbol) [0x00007FF6424518DC]
	(No symbol) [0x00007FF64244819C]
	(No symbol) [0x00007FF6424720EF]
	(No symbol) [0x00007FF6424480CF]
	(No symbol) [0x00007FF6424722C0]
	(No symbol) [0x00007FF64248AAA4]
	(No symbol) [0x00007FF642471E83]
	(No symbol) [0x00007FF64244670A]
	(No symbol) [0x00007FF642447964]
	GetHandleVerifier [0x00007FF642960AAB+3694587]
	GetHandleVerifier [0x00007FF6429B728E+4048862]
	GetHandleVerifier [0x00007FF6429AF173+4015811]
	GetHandleVerifier [0x00007FF6426847D6+69559

Unificación

Este es el que completé que da la información de los hoteles

In [8]:
def obtener_atributos(url,contador):
    # Instanciate and configurate driver
    chrome_options = Options()
    chrome_options.add_argument('--headless') # unables GUI
    chrome_options.add_argument('--disable-infobars') # unables images loading
    driver = webdriver.Chrome(options=chrome_options)
    
    try:
        # Abrir la URL proporcionada
        driver.get(url)
        time.sleep(5)

        # Obtener dirección
        try:
            direccion_element = driver.find_element(By.CLASS_NAME, "hp_address_subtitle")
            direccion_texto = direccion_element.text
        except:
            direccion_texto = None

        # Obtener descripción
        try:
            descripcion_element = driver.find_element(By.CLASS_NAME, "b3efd73f69")
            descripcion_texto = descripcion_element.text
        except:
            descripcion_texto = None

        # Obtener atributos
        try:
            list_items = driver.find_elements(By.XPATH, "/html/body/div[3]/div/div[5]/div[1]/div[1]/div[3]/div/div/div[1]/div[2]/div[2]/div/div/ul/li")
            attributes_list = [li.text for li in list_items]
        except:
            attributes_list = None

        # Obtener latitud y longitud
        try:
            map_element = driver.find_element(By.ID, 'hotel_sidebar_static_map')
            lat_lng_attribute = map_element.get_attribute('data-atlas-latlng')
            lat, lng = map(float, lat_lng_attribute.split(','))
        except:
            lat, lng = None, None

    finally:
        # Cerrar el navegador
        driver.quit()

    # Crear el DataFrame
    df_atributos = pd.DataFrame({
        "Dirección": [direccion_texto],
        "Descripción": [descripcion_texto],
        "Atributos": [attributes_list],
        "Latitud": [lat],
        "Longitud": [lng]
    })
    

    return df_atributos

In [10]:
# Ejemplo de uso
df_atributos_resultante = pd.DataFrame()

for contador, (index, row) in enumerate(data.iterrows(), start=1):
    url_ejemplo = row["url"]
    # Imprimir el progreso
    print(f"Scrapeando registro {contador} de {len(data)}...")
    df_atributos_resultante = pd.concat([df_atributos_resultante, obtener_atributos(url_ejemplo, contador)], ignore_index=True)

# Guardar el DataFrame resultante en un archivo CSV
#df_atributos_resultante.to_csv('atributos_resultantes.csv', index=False)


Scrapeando registro 1 de 1290...
Scrapeando registro 2 de 1290...
Scrapeando registro 3 de 1290...
Scrapeando registro 4 de 1290...
Scrapeando registro 5 de 1290...
Scrapeando registro 6 de 1290...
Scrapeando registro 7 de 1290...
Scrapeando registro 8 de 1290...
Scrapeando registro 9 de 1290...
Scrapeando registro 10 de 1290...
Scrapeando registro 11 de 1290...
Scrapeando registro 12 de 1290...
Scrapeando registro 13 de 1290...
Scrapeando registro 14 de 1290...
Scrapeando registro 15 de 1290...
Scrapeando registro 16 de 1290...
Scrapeando registro 17 de 1290...
Scrapeando registro 18 de 1290...
Scrapeando registro 19 de 1290...
Scrapeando registro 20 de 1290...
Scrapeando registro 21 de 1290...
Scrapeando registro 22 de 1290...
Scrapeando registro 23 de 1290...
Scrapeando registro 24 de 1290...
Scrapeando registro 25 de 1290...
Scrapeando registro 26 de 1290...
Scrapeando registro 27 de 1290...
Scrapeando registro 28 de 1290...
Scrapeando registro 29 de 1290...
Scrapeando registro 30 

: 

In [94]:
# Ejemplo de uso para los primeros 10 enlaces
df_atributos_resultante = pd.DataFrame()

# Tomar solo los primeros 10 enlaces
for index, row in data.head(10).iterrows():
    url_ejemplo = row["url"]
    df_atributos_resultante = pd.concat([df_atributos_resultante, obtener_atributos(url_ejemplo)], ignore_index=True)

print(df_atributos_resultante.head())

                                           Dirección  \
0  500 South Main Street, Salt Lake City, UT 8410...   
1  170 South West Temple, Salt Lake City, 84101, ...   
2  171 West 500 South, Salt Lake City, UT 84101, ...   
3  206 South West Temple, Salt Lake City, UT 8410...   
4  423 West 300 South, Salt Lake City, UT 84101, ...   

                                         Descripción  \
0  Este hotel de Salt Lake City se encuentra a 5 ...   
1  Hyatt Regency Salt Lake City es un alojamiento...   
2  El hotel Comfort Inn Downtown está en el centr...   
3  El Holiday Inn Express Salt Lake City Downtown...   
4  Este hotel se encuentra en el centro de Salt L...   

                                           Atributos    Latitud    Longitud  
0  [2 piscinas (1 abierta), Habitaciones sin humo...  40.758264 -111.891465  
1  [Piscina al aire libre, Habitaciones sin humo,...  40.766077 -111.894710  
2  [Habitaciones sin humo, Gimnasio, WiFi gratis,...  40.758091 -111.895454  
3  [Piscina in

In [95]:
df_atributos_resultante

,Dirección,Descripción,Atributos,Latitud,Longitud
0,"500 South Main Street, Salt Lake City, UT 8410...",Este hotel de Salt Lake City se encuentra a 5 ...,"[2 piscinas (1 abierta), Habitaciones sin humo...",40.758264,-111.891465
1,"170 South West Temple, Salt Lake City, 84101, ...",Hyatt Regency Salt Lake City es un alojamiento...,"[Piscina al aire libre, Habitaciones sin humo,...",40.766077,-111.894710
2,"171 West 500 South, Salt Lake City, UT 84101, ...",El hotel Comfort Inn Downtown está en el centr...,"[Habitaciones sin humo, Gimnasio, WiFi gratis,...",40.758091,-111.895454
3,"206 South West Temple, Salt Lake City, UT 8410...",El Holiday Inn Express Salt Lake City Downtown...,"[Piscina interior, Habitaciones sin humo, Adap...",40.764598,-111.893959
4,"423 West 300 South, Salt Lake City, UT 84101, ...",Este hotel se encuentra en el centro de Salt L...,"[Piscina al aire libre, Habitaciones sin humo,...",40.762626,-111.903379
5,"150 North Main Street, Salt Lake City, UT 8410...",Apartments feature a full living room. The Kim...,"[Habitaciones sin humo, Gimnasio, WiFi gratis,...",40.772699,-111.890571
6,"223 Jimmy Doolittle, Salt Lake City, UT 84116,...",This Utah hotel is 10 minutes' drive from down...,"[Piscina interior, Habitaciones sin humo, Tras...",40.772571,-112.020341
7,"176 W 600 S, Salt Lake City, UT 84101, Estados...","Este motel de Salt Lake City, Utah, está a sol...","[Piscina al aire libre, Habitaciones sin humo,...",40.756642,-111.896426
8,"1009 South Main Street, Salt Lake City, UT 841...",Este hotel de Utah se encuentra a 15 minutos e...,"[Piscina al aire libre, Habitaciones sin humo,...",40.747054,-111.890751
9,"434 East 1700 South, Salt Lake City, UT 84115,...","Situated in Salt Lake City, 5.4 km from The Ta...","[WiFi gratis, Parking gratis]",40.733318,-111.878786


In [89]:
# Ejemplo de uso
df_atributos_resultante = pd.DataFrame()

for index, row in data.iterrows():
    url_ejemplo = row["url"]
    df_atributos_resultante = pd.concat([df_atributos_resultante, obtener_atributos(url_ejemplo)], ignore_index=True)

print(df_atributos_resultante.head())

SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location C:\Program Files (x86)\Google\Chrome\Application\chrome.exe is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
	GetHandleVerifier [0x00007FF6425E82B2+55298]
	(No symbol) [0x00007FF642555E02]
	(No symbol) [0x00007FF6424105AB]
	(No symbol) [0x00007FF64243DCD8]
	(No symbol) [0x00007FF64243B30E]
	(No symbol) [0x00007FF64247A198]
	(No symbol) [0x00007FF642479CB0]
	(No symbol) [0x00007FF642471E83]
	(No symbol) [0x00007FF64244670A]
	(No symbol) [0x00007FF642447964]
	GetHandleVerifier [0x00007FF642960AAB+3694587]
	GetHandleVerifier [0x00007FF6429B728E+4048862]
	GetHandleVerifier [0x00007FF6429AF173+4015811]
	GetHandleVerifier [0x00007FF6426847D6+695590]
	(No symbol) [0x00007FF642560CE8]
	(No symbol) [0x00007FF64255CF34]
	(No symbol) [0x00007FF64255D062]
	

### Dirección

In [45]:
# Instanciate and configurate driver
chrome_options = Options()
#chrome_options.add_argument('--headless') # unables GUI
chrome_options.add_argument('--disable-infobars') # unables images loading
driver = webdriver.Chrome(options=chrome_options)

url = data["url"][0]
driver.get(url)
time.sleep(5)

# Encontrar el elemento por clase
direccion_element = driver.find_element(By.CLASS_NAME,"hp_address_subtitle")

# Obtener el texto del elemento
direccion_texto = direccion_element.text

# Imprimir la dirección
print("Dirección:", direccion_texto)

# Cerrar el navegador
driver.quit()

Dirección: 500 South Main Street, Salt Lake City, UT 84101, Estados Unidos


### Descripción

In [46]:
# Instanciate and configurate driver
chrome_options = Options()
#chrome_options.add_argument('--headless') # unables GUI
chrome_options.add_argument('--disable-infobars') # unables images loading
driver = webdriver.Chrome(options=chrome_options)

url = data["url"][0]
driver.get(url)
time.sleep(5)

# Encontrar el elemento por clase
descripcion_element = driver.find_element(By.CLASS_NAME, "b3efd73f69")

# Obtener el texto del elemento
descripcion_texto = descripcion_element.text

# Imprimir la descripción
print("Descripción:", descripcion_texto)

# Cerrar el navegador
driver.quit()

Descripción: Este hotel de Salt Lake City se encuentra a 5 minutos en coche de la plaza Temple. Ofrece piscina cubierta, piscina al aire libre y tiendas. Los domingos se sirve un brunch que ha sido galardonado.

Las habitaciones del Little America Hotel disponen de TV y WiFi gratuita. Las habitaciones superiores Tower incluyen zona de estar independiente, TV de pantalla plana y soporte para iPod.

El spa del Little American alberga sauna, bañera de hidromasaje y centro de fitness. Además, hay 2 tiendas, una de ellas de regalos.

El Lucky H Bar & Grille sirve desayunos y almuerzos buffet, así como un menú a la carta para la cena. La cafetería prepara cocina tradicional en un entorno informal. El hotel también alberga una cafetería y dispone de servicio de habitaciones.

El aeropuerto internacional de Salt Lake City se halla a 10 minutos en coche. Además, la Universidad de Utah está a 4,7 km del Little America Hotel Salt Lake City.


### Atributos

In [51]:
# Instanciate and configurate driver
chrome_options = Options()
#chrome_options.add_argument('--headless') # unables GUI
chrome_options.add_argument('--disable-infobars') # unables images loading
driver = webdriver.Chrome(options=chrome_options)

url = data["url"][0]
driver.get(url)
time.sleep(5)

# Encontrar los elementos li directamente
list_items = driver.find_elements(By.XPATH, "/html/body/div[3]/div/div[5]/div[1]/div[1]/div[3]/div/div/div[1]/div[2]/div[2]/div/div/ul/li")

# Crear una lista para guardar los atributos
attributes_list = []

for li in list_items:
    # Extraer el contenido de los elementos li
    attributes_list.append(li.text)

print(attributes_list)

# Cerrar el navegador
driver.quit()

['2 piscinas (1 abierta)', 'Habitaciones sin humo', '2 restaurantes', 'Adaptado personas de movilidad reducida', 'Gimnasio', 'Servicio de habitaciones', 'WiFi gratis', 'Parking en el alojamiento', 'Tetera/cafetera en todas las habitaciones', 'Bar']


### Latitud y longitud del mapa

In [ ]:
# Instanciate and configurate driver
chrome_options = Options()
#chrome_options.add_argument('--headless') # unables GUI
chrome_options.add_argument('--disable-infobars') # unables images loading
driver = webdriver.Chrome(options=chrome_options)

url = data["url"][0]
driver.get(url)
time.sleep(5)

# Encontrar el elemento por su ID
map_element = driver.find_element(By.ID, 'hotel_sidebar_static_map')

# Obtener el atributo 'data-atlas-latlng' que contiene la latitud y longitud
lat_lng_attribute = map_element.get_attribute('data-atlas-latlng')

# Separar la latitud y longitud
lat, lng = map(float, lat_lng_attribute.split(','))

# Imprimir los resultados
print('Latitud:', lat)
print('Longitud:', lng)

# Cerrar el navegador
driver.quit()

Latitud: 40.75826405
Longitud: -111.89146487


## Links para scrapear atracciones

### West Mountain:

- Arizona - AZ        -- URL = "https://www.tripadvisor.com.ar/Attractions-g28924-Activities-oa0-Arizona.html"
- Colorado - CO       -- URL = "https://www.tripadvisor.com.ar/Attractions-g28927-Activities-oa0-Colorado.html"
- Idaho - ID          -- URL = "https://www.tripadvisor.com.ar/Attractions-g28933-Activities-oa0-Idaho.html"
- Montana - MT        -- URL = "https://www.tripadvisor.com.ar/Attractions-g28947-Activities-oa0-Montana.html"
- Nevada - NV         -- URL = "https://www.tripadvisor.com.ar/Attractions-g28949-Activities-oa0-Nevada.html"
- Nuevo México - NM   -- URL = "https://www.tripadvisor.com.ar/Attractions-g28952-Activities-oa0-New_Mexico.html"
- Utah - UT           -- URL = "https://www.tripadvisor.com.ar/Attractions-g28965-Activities-oa0-Utah.html"
- Wyoming - WY        -- URL = "https://www.tripadvisor.com.ar/Attractions-g28973-Activities-oa0-Wyoming.html"

### South:

#### West South Central:

- Arkansas - AR       -- URL = "https://www.tripadvisor.com.ar/Attractions-g28925-Activities-oa0-Arkansas.html"
- Luisiana - LA       -- URL = "https://www.tripadvisor.com.ar/Attractions-g28939-Activities-oa0-Louisiana.html"
- Oklahoma - OK       -- URL = "https://www.tripadvisor.com.ar/Attractions-g28957-Activities-oa0-Oklahoma.html"
- Texas - TX          -- URL = "https://www.tripadvisor.com.ar/Attractions-g28964-Activities-oa0-Texas.html"

#### East South Central:

- Alabama - AL        -- URL = "https://www.tripadvisor.com.ar/Attractions-g28922-Activities-oa0-Alabama.html"
- Kentucky - KY       -- URL = "https://www.tripadvisor.com.ar/Attractions-g28938-Activities-oa0-Kentucky.html"
- Mississipi - MS       -- URL = "https://www.tripadvisor.com.ar/Attractions-g28945-Activities-oa0-Mississippi.html"
- Tennessee - TN      -- URL = "https://www.tripadvisor.com.ar/Attractions-g28963-Activities-oa0-Tennessee.html"

#### South Atlantic:

- Delaware - DE             -- URL = "https://www.tripadvisor.com.ar/Attractions-g28929-Activities-oa0-Delaware.html"
- Florida - FL              -- URL = "https://www.tripadvisor.com.ar/Attractions-g28930-Activities-oa0-Florida.html"
- Georgia - GA              -- URL = "https://www.tripadvisor.com.ar/Attractions-g28931-Activities-oa0-Georgia.html"
- Carolina del Sur - SC     -- URL = "https://www.tripadvisor.com.ar/Attractions-g28961-Activities-oa0-South_Carolina.html"
- Carolina del Norte - NC   -- URL = "https://www.tripadvisor.com.ar/Attractions-g28954-Activities-oa0-North_Carolina.html"
- Maryland - MD             -- URL = "https://www.tripadvisor.com.ar/Attractions-g28941-Activities-oa0-Maryland.html"
- Virginia - VA             -- URL = "https://www.tripadvisor.com.ar/Attractions-g28967-Activities-oa0-Virginia.html"
- Virginia Occidental - WV  -- URL = "https://www.tripadvisor.com.ar/Attractions-g28971-Activities-oa0-West_Virginia.html"
- Washington, D.C. - DC     -- URL = "https://www.tripadvisor.com.ar/Attractions-g28970-Activities-oa0-Washington_DC_District_of_Columbia.html"

### Midwest:

#### West North Central:

- Dakota del Norte - ND     -- URL = ""
- Dakota del Sur - SD       -- URL = ""
- Iowa - IA                 -- URL = ""
- Kansas - KS               -- URL = ""
- Minnesota - MN            -- URL = ""
- Missouri - MO             -- URL = ""
- Nebraska - NE             -- URL = ""
- Wyoming - WY              -- URL = ""

#### East North Central:

- Illinois - IL       -- URL = ""
- Indiana - IN        -- URL = ""
- Míchigan - MI       -- URL = ""
- Ohio - OH           -- URL = ""
- Wisconsin - WI      -- URL = ""

### Northeast:

#### New England:

- Connecticut - CT      -- URL = ""
- Maine - ME            -- URL = ""
- Massachusetts - MA    -- URL = ""
- Nuevo Hampshire - NH  -- URL = ""
- Rhode Island - RI     -- URL = ""
- Vermont - VT          -- URL = ""

#### Mid Atlantic:

- Nueva Jersey - NJ         -- URL = ""
- Nueva York - NY           -- URL = ""
- Pensilvania - PA          -- URL = ""
- Delaware - DE             -- URL = ""
- Maryland - MD             -- URL = ""
- Washington, D.C. - DC     -- URL = ""
